In [3]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import BertModel
from transformers import AutoModel
import torch

tokenizer = BertTokenizer.from_pretrained('BERT-pubmed-1000000-SQuAD')
model = BertForQuestionAnswering.from_pretrained("BERT-pubmed-1000000-SQuAD")

I0529 03:54:18.534895 4642352576 tokenization_utils.py:937] Model name 'BERT-pubmed-1000000-SQuAD' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'BERT-pubmed-1000000-SQuAD' is a path, a model identifier, or url to a directory containing tokenizer files.
I0529 03:54:18.536962 4642352576 tokenization_utils.py:966] Didn't find file BERT-pubmed-1000000-SQuAD/added_tokens.json. We won't load it.
I0529 03:54:18.544423 4642352576 tokenization_utils.py:966] Didn'

In [4]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    
    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    print(len(start_scores.detach().numpy().flatten()))
    print(len(tokens))

    # Start with the first token.
    answer = tokens[answer_start]
    
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [463]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = '''
Attending:[**First Name3 (LF) 148**]
Chief Complaint:
Abdominal Pain

History of Present Illness:
This 48-year-old [**First Name3 (LF) 1229**] has mental retardation and a seizure
syndrome. He presents to our emergency room acutely with reports
of [**1-15**] days of abdominal
pain as described by his caretakers, who find him grimacing in
an umbilical position. He had a change in bowel habits and
decreased PO intake for 2 weeks. He is largely unresponsive and
he responds only to keep the stimulation for pain. He has had
fevers for the last few days, up as high as 104 degrees. A
workup was performed for this, and initial imaging of the
abdomen showed multiple views consistent with a free air in the
abdomen. This with a lactic acidosis, distended abdomen and a
neutrophilia band shift, along with the after mentioned history
was very concerning for an acute process which
required an emergent operation. This was especially so given the
fact that we could not adequately communicate with this
[**Name2 (NI) 1229**] and did not know the full extent of his recognition of
pain due to his mental retardation.
'''


print(wrapper.fill(bert_abstract))


 Attending:[**First Name3 (LF) 148**] Chief Complaint: Abdominal Pain  History
of Present Illness: This 48-year-old [**First Name3 (LF) 1229**] has mental
retardation and a seizure syndrome. He presents to our emergency room acutely
with reports of [**1-15**] days of abdominal pain as described by his
caretakers, who find him grimacing in an umbilical position. He had a change in
bowel habits and decreased PO intake for 2 weeks. He is largely unresponsive and
he responds only to keep the stimulation for pain. He has had fevers for the
last few days, up as high as 104 degrees. A workup was performed for this, and
initial imaging of the abdomen showed multiple views consistent with a free air
in the abdomen. This with a lactic acidosis, distended abdomen and a
neutrophilia band shift, along with the after mentioned history was very
concerning for an acute process which required an emergent operation. This was
especially so given the fact that we could not adequately communicate with this

In [ ]:
question = "What is chief complaint of a patient?"

answer_question(question, bert_abstract)